In [3]:
import os
import sys
module_path = os.path.abspath( os.pardir)
if module_path not in sys.path:
    sys.path.append(module_path)
from src import data_download

In [4]:
tables = data_download.get_tables(get_dataframes())

AttributeError: module 'src.data_download' has no attribute 'get_tables'